## 数据预处理

In [1]:
import os
import cv2
import h5py
import numpy as np
import pandas as pd
from keras.applications import Xception, xception
from keras.models import Model
from keras.layers import Dropout, Dense, Input, Lambda
from keras.callbacks import Callback
from sklearn.model_selection import train_test_split
from tqdm import tqdm

data_path_train = '../dogs-vs-cats-dataset/train'
data_path_test = '../dogs-vs-cats-dataset/test'
image_names_train = os.listdir(data_path_train)
image_names_test = os.listdir(data_path_test)
input_shape = (299, 299, 3)

print('Done!')

Using TensorFlow backend.


Done!


In [2]:
labels = []
trains = []
tests = []

# 处理标准的训练数据
for i in tqdm(range(len(image_names_train))):
    image_name = image_names_train[i]
    image_path = os.path.join(data_path_train, image_name)
    image = cv2.imread(image_path)
    if image is None:
        print('Read train image failed:', image_path)
        continue
    image = cv2.resize(image, (input_shape[0], input_shape[1]))
    trains.append(image[:, :, ::-1])
    # cat: 0, dog: 1
    category = 1 if 'dog' in image_name else 0
    labels.append(category)


# 处理标准的测试数据
for i in tqdm(range(len(image_names_test))):
    image_name = image_names_test[i]
    image_path = os.path.join(data_path_test, image_name)
    image = cv2.imread(image_path)
    if image is None:
        print('Read test image failed:', image_path)
        continue
    image = cv2.resize(image, (input_shape[0], input_shape[1]))
    tests.append(image[:, :, ::-1])
    
    
trains = np.array(trains)
labels = np.array(labels)
tests = np.array(tests)

print('Training data size: %d' % len(trains))
print('Label size: %d' % len(labels))
print('Testing data size: %d' % len(tests))

100%|██████████| 12500/12500 [00:28<00:00, 443.18it/s]


Training data size: 25000
Label size: 25000
Testing data size: 12500


## 特征提取

In [3]:
x = Input(shape=input_shape)
x = Lambda(xception.preprocess_input)(x)
model = Xception(input_tensor=x, input_shape=input_shape, weights='imagenet', include_top=False, pooling='avg')
bottleneck_features_train = model.predict(trains, batch_size=128)
bottleneck_features_test = model.predict(tests, batch_size=128)

with h5py.File("bottleneck_features.h5", 'w') as h:
    h.create_dataset('trains', data=bottleneck_features_train)
    h.create_dataset('labels', data=labels)
    h.create_dataset('tests', data=bottleneck_features_test)

print('bottleneck features have been wrote to bottleneck_features.h5')

bottleneck features have been wrote to bottleneck_features.h5


## 构建模型

In [2]:
with h5py.File('bottleneck_features.h5','r') as h:
    X_train = np.array(h['trains'])
    y_train = np.array(h['labels'])
    X_test = np.array(h['tests'])

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, shuffle=True, test_size=0.2, random_state=2018)

def getModel(drop_rate=0.5):  
    x = Input(shape=(X_train.shape[1],))
    y = Dropout(0.2)(x)
    y = Dense(1, activation='sigmoid')(y)
    mod = Model(x, y)
    mod.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])
    return mod

print('Model ready!')

Model ready!


## 训练与优化

### 预测函数

In [3]:
# 在测试数据集上预测结果，且保存在.csv文件中
def predict_func(mod, file_name):
    y_pred = mod.predict(X_test, verbose=1)
    y_pred = y_pred.clip(min=0.005, max=0.995)

    df = pd.read_csv("sample_submission.csv")

    for i in range(len(image_names_test)):
        image_name = image_names_test[i]
        index = int(str.split(image_name, '.')[0]) - 1
        df.iat[index, 1] = y_pred[i]

    df.to_csv(os.path.join('./predict-csv', file_name), index=None)
    print('The prediction result has been wrote to: ', file_name)

### 回调函数

In [4]:
# 用于优化Epochs参数的回调函数
class EpochsLossCallback(Callback):
    def on_epoch_end(self, epoch, logs={}):
        predict_func(self.model, 'predict' + '_epoch' + str(epoch + 1) + '.csv')

### 模型优化之——Epochs

In [8]:
model = getModel(drop_rate=0.2)
model.fit(
    x=X_train,
    y=y_train,
    batch_size=128,
    epochs=20,
    validation_data=(X_val, y_val),
    callbacks=[EpochsLossCallback()]
)

Train on 20000 samples, validate on 5000 samples
Epoch 1/20
12500/12500 [==============================] - 0s 39us/step
The prediction result has been wrote to:  predict_epoch1.csv
Epoch 2/20
12500/12500 [==============================] - 0s 27us/step
The prediction result has been wrote to:  predict_epoch2.csv
Epoch 3/20
12500/12500 [==============================] - 0s 27us/step
The prediction result has been wrote to:  predict_epoch3.csv
Epoch 4/20
12500/12500 [==============================] - 0s 27us/step
The prediction result has been wrote to:  predict_epoch4.csv
Epoch 5/20
12500/12500 [==============================] - 0s 27us/step
The prediction result has been wrote to:  predict_epoch5.csv
Epoch 6/20
12500/12500 [==============================] - 0s 27us/step
The prediction result has been wrote to:  predict_epoch6.csv
Epoch 7/20
12500/12500 [==============================] - 0s 26us/step
The prediction result has been wrote to:  predict_epoch7.csv
Epoch 8/20
12500/12500 [===

结论：Epochs选9为最佳

### 模型优化之——drop rate

In [5]:
for i in range(10):
    drop_rate = round(0.1 * i, 2)
    model = getModel(drop_rate=drop_rate)
    model.fit(
        x=X_train,
        y=y_train,
        batch_size=128,
        epochs=9,
        validation_data=(X_val, y_val))
    predict_func(model, 'predict' + '_drop_rate_' + str(drop_rate) + '.csv')
    
print('Done!')

Train on 20000 samples, validate on 5000 samples
Epoch 1/9
20000/20000 [==============================] - 1s 73us/step - loss: 0.1452 - acc: 0.9744 - val_loss: 0.0485 - val_acc: 0.9910
Epoch 2/9
20000/20000 [==============================] - 1s 25us/step - loss: 0.0338 - acc: 0.9928 - val_loss: 0.0299 - val_acc: 0.9916
Epoch 3/9
20000/20000 [==============================] - 1s 25us/step - loss: 0.0231 - acc: 0.9939 - val_loss: 0.0260 - val_acc: 0.9924
Epoch 4/9
20000/20000 [==============================] - 1s 26us/step - loss: 0.0193 - acc: 0.9944 - val_loss: 0.0249 - val_acc: 0.9922
Epoch 5/9
20000/20000 [==============================] - 1s 26us/step - loss: 0.0173 - acc: 0.9949 - val_loss: 0.0242 - val_acc: 0.9922
Epoch 6/9
20000/20000 [==============================] - 1s 25us/step - loss: 0.0163 - acc: 0.9950 - val_loss: 0.0238 - val_acc: 0.9926
Epoch 7/9
20000/20000 [==============================] - 1s 25us/step - loss: 0.0159 - acc: 0.9950 - val_loss: 0.0247 - val_acc: 0.9930

12500/12500 [==============================] - 0s 26us/step
The prediction result has been wrote to:  predict_drop_rate_0.5.csv
Train on 20000 samples, validate on 5000 samples
Epoch 1/9
20000/20000 [==============================] - 1s 39us/step - loss: 0.1352 - acc: 0.9816 - val_loss: 0.0474 - val_acc: 0.9912
Epoch 2/9
20000/20000 [==============================] - 1s 25us/step - loss: 0.0335 - acc: 0.9928 - val_loss: 0.0299 - val_acc: 0.9924
Epoch 3/9
20000/20000 [==============================] - 1s 25us/step - loss: 0.0229 - acc: 0.9940 - val_loss: 0.0262 - val_acc: 0.9926
Epoch 4/9
20000/20000 [==============================] - 1s 25us/step - loss: 0.0198 - acc: 0.9943 - val_loss: 0.0247 - val_acc: 0.9922
Epoch 5/9
20000/20000 [==============================] - 1s 25us/step - loss: 0.0178 - acc: 0.9943 - val_loss: 0.0244 - val_acc: 0.9926
Epoch 6/9
20000/20000 [==============================] - 1s 25us/step - loss: 0.0163 - acc: 0.9950 - val_loss: 0.0236 - val_acc: 0.9922
Epoch 7

结论：drop rate选择0.9

## 最终的训练和预测

In [58]:
model = getModel(drop_rate=0.9)
    model.fit(
        x=X_train,
        y=y_train,
        batch_size=128,
        epochs=9,
        validation_data=(X_val, y_val))
    predict_func(model, 'predict.csv')

12500/12500 [==============================] - 1s 82us/step
The prediction result has been wrote to predict.csv
